# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [53]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD

from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine(r'sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM classified_msgs', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
list(df.columns)

['id',
 'message',
 'original',
 'genre',
 'related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [4]:
X = df['message']
Y = df.iloc[:,4:]

In [5]:
list(Y.columns)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    # prep nltk transformation objects
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmed = [lemmatizer.lemmatize(word, pos='v') for word in tokens if word not in stop_words]
    
    # Reduce words to their stems
    stemmed = [stemmer.stem(word) for word in lemmed]

    return stemmed

Test the tokenize function:

In [7]:
for i in range(10):
    print(X[i])
    print(tokenize(X[i]))
    print("")
    print('--------------------------')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

--------------------------
Is the Hurricane over or is it not over
['hurrican']

--------------------------
Looking for someone but no name
['look', 'someon', 'name']

--------------------------
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']

--------------------------
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']

--------------------------
Information about the National Palace-
['inform', 'nation', 'palac']

--------------------------
Storm at sacred heart of jesus
['storm', 'sacr', 'heart', 'jesu']

--------------------------
Please, we need tents and water. We are in Silo, Thank you!
['plea

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect' , CountVectorizer(tokenizer=tokenize)),
        ('tfidf' , TfidfTransformer()),
        ('clf' , MultiOutputClassifier(AdaBoostClassifier(random_state = 42)))
], verbose = True)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [10]:
pipeline.fit(X_train, Y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=  23.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.1min


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))],
         verbose=True)

### EXTRA: Naive predictions

To understand how much improvement the ML model brings, let's first see what is the baseline Naive accuracy, which would be achieved by predicting the majority class for each category

In [11]:
naive_pred = Y_test.copy()
for category in Y_test.columns:
    
    # check accuracy of predicting TRUE to all
    categ_acc = Y_test[category].mean()
    
    # predict the majority class:
    if categ_acc>0.5:
        naive_pred[category] = 1
    else:
        naive_pred[category] = 0

In [12]:
# create other auxiliary functions:

In [13]:
def average_accuracy(Y_real, Y_predictions):
    '''
    Computes average accuracy across all categories:
    
    inputs:
    Y_real: correct classifications
    Y_predictions: predicted classifications
    
    Return:
    accuracy
    '''
    
    return (Y_real == Y_predictions).sum().sum()/Y_real.size

In [14]:
print("Naive accuracy: ", average_accuracy(Y_test, naive_pred))

Naive accuracy:  0.9271691587834401


In [56]:
def average_precision(Y_real, Y_predictions, avg = 'macro'):
    '''
    Computes average precision across all categories:
    
    inputs:
    Y_real: correct classifications
    Y_predictions: predicted classifications
    avg: Default == 'macro'. See f1_score doc for other options
    
    Return:
    accuracy
    '''
    
    prec = []
    
    # iterate
    for i, category in enumerate(Y_real):
        prec.append(precision_score(Y_real[category], Y_predictions[:, i], average = avg))
        
    
    return np.mean(prec)

In [58]:
print("Naive precision: ", average_precision(Y_test, naive_pred.to_numpy()))

Naive precision:  0.47747346828060905


C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
def average_recall(Y_real, Y_predictions, avg = 'macro'):
    '''
    Computes average recall across all categories:
    
    inputs:
    Y_real: correct classifications
    Y_predictions: predicted classifications
    avg: Default == 'macro'. See f1_score doc for other options
    
    Return:
    accuracy
    '''
    
    rec = []
    
    # iterate
    for i, category in enumerate(Y_real):
        rec.append(recall_score(Y_real[category], Y_predictions[:, i], average = avg))
        
    
    return np.mean(rec)

In [61]:
print("Naive recall: ", average_recall(Y_test, naive_pred.to_numpy()))

Naive recall:  0.5138888888888888


In [15]:
def average_f1(Y_real, Y_predictions, avg = 'macro'):
    '''
    Computes average f1-score across all categories:
    
    inputs:
    Y_real: correct classifications
    Y_predictions: predicted classifications
    avg: Default == 'macro'. See f1_score doc for other options
    
    Return:
    accuracy
    '''
    
    f1 = []
    
    # iterate
    for i, category in enumerate(Y_real):
        f1.append(f1_score(Y_real[category], Y_predictions[:, i], average = avg))
        
    
    return np.mean(f1)

In [16]:
print("Naive f1-score: ", average_f1(Y_test, naive_pred.to_numpy()))

Naive f1-score:  0.49381425354511244


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
Y_pred = pipeline.predict(X_test)

In [20]:
def print_classification_report(Y_real, Y_predictions):
    '''
    Prints classification report per category:
    
    inputs:
    Y_real: correct classifications
    Y_predictions: predicted classifications
    
    Return:
    none
    '''
    # create lists:
    acc = []
    f1 = []
    
    # iterate
    for i, category in enumerate(Y_real):
        print("Category: ",category)
        print(classification_report(Y_real[category], Y_predictions[:, i]))
        f1.append(f1_score(Y_real[category], Y_predictions[:, i], average = 'macro'))
        acc.append(accuracy_score(Y_real[category], Y_predictions[:, i]))

    print("-------------------------------------------------------")    
    print("Mean accuracy score: {:.4f}".format(np.mean(acc)))
    print("Mean f1-score (macro): {:.4f}".format(np.mean(f1)))

In [21]:
print_classification_report(Y_test, Y_pred)

Category:  related
              precision    recall  f1-score   support

           0       0.68      0.24      0.35      1553
           1       0.80      0.97      0.88      5001

    accuracy                           0.79      6554
   macro avg       0.74      0.60      0.62      6554
weighted avg       0.77      0.79      0.75      6554

Category:  request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5461
           1       0.77      0.52      0.62      1093

    accuracy                           0.89      6554
   macro avg       0.84      0.74      0.78      6554
weighted avg       0.89      0.89      0.88      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       

Category:  other_weather
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6235
           1       0.44      0.16      0.24       319

    accuracy                           0.95      6554
   macro avg       0.70      0.58      0.61      6554
weighted avg       0.93      0.95      0.94      6554

Category:  direct_report
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      5289
           1       0.70      0.38      0.50      1265

    accuracy                           0.85      6554
   macro avg       0.79      0.67      0.70      6554
weighted avg       0.84      0.85      0.83      6554

-------------------------------------------------------
Mean accuracy score: 0.9479
Mean f1-score (macro): 0.6838


In [22]:
print("Naive accuracy: ", average_accuracy(Y_test, naive_pred))
print("Base model accuracy: ", average_accuracy(Y_test, Y_pred))

Naive accuracy:  0.9271691587834401
Base model accuracy:  0.947919845387041


In [62]:
print("Naive precision: ", average_precision(Y_test, naive_pred.to_numpy()))
print("Base model precision: ", average_precision(Y_test, Y_pred))

Naive precision:  0.47747346828060905
Base model precision:  0.7637077465683915


In [63]:
print("Naive recall: ", average_recall(Y_test, naive_pred.to_numpy()))
print("Base model recall: ", average_recall(Y_test, Y_pred))

Naive recall:  0.5138888888888888
Base model recall:  0.6550423293297105


In [23]:
print("Naive f1-score: ", average_f1(Y_test, naive_pred.to_numpy()))
print("Base model f1-score: ", average_f1(Y_test, Y_pred))

Naive f1-score:  0.49381425354511244
Base model f1-score:  0.6837673978011033


Check some rows:

In [24]:
#define styler
def is_equal(row):
    is_equal = row[0]==row[1]
    
    if is_equal:
        return ['background-color: lightgreen','background-color: lightgreen' ]
    else:
        return ['background-color: red','background-color: red'] 

# define function to compare actual with prediction:
def check_row(i):
    print("original msg:")
    print(X_test.iloc[i])
    print("")
    print("tokenized msg:")
    print(tokenize(X_test.iloc[i]))
    print("")
    print("Accuracy: ", accuracy_score(Y_test.iloc[i,], Y_pred[i,]))
    print("")
    print(pd.DataFrame(precision_recall_fscore_support(Y_test.iloc[i,], Y_pred[i,]),
                       index = ['precision','recall', 'f1','support']))
    print("")

    check = pd.DataFrame(np.column_stack([Y_test.iloc[i,], Y_pred[i,]]), index = Y_test.columns, columns = ['Real','Pred'])
    return check.style.apply(is_equal, axis = 1)

In [25]:
check_row(1)

original msg:
Rehabilitation and restoration of damaged water and sanitation facilities: This includes for the procurement of supplies and equipment, electric pumps, handpumps, spare parts pipes, electrical switchgear, piping, cabling, and so on, to replace those damaged in the flooding, for all costs associated with labour, transport and other operational costs associated with the rehabilitation of these facilities and where appropriate for the procurement of technical assistance services.

tokenized msg:
['rehabilit', 'restor', 'damag', 'water', 'sanit', 'facil', 'includ', 'procur', 'suppli', 'equip', 'electr', 'pump', 'handpump', 'spare', 'part', 'pip', 'electr', 'switchgear', 'pip', 'cabl', 'replac', 'damag', 'flood', 'cost', 'associ', 'labour', 'transport', 'oper', 'cost', 'associ', 'rehabilit', 'facil', 'appropri', 'procur', 'technic', 'assist', 'servic']

Accuracy:  0.9444444444444444

                   0         1
precision   0.966667  0.833333
recall      0.966667  0.833333
f

,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,1,0
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


In [26]:
check_row(20)

original msg:
So they re nw gonna use their alreadi inefficient state funds 2 deal with the aftamath of da earthquake

tokenized msg:
['nw', 'gon', 'na', 'use', 'alreadi', 'ineffici', 'state', 'fund', '2', 'deal', 'aftamath', 'da', 'earthquak']

Accuracy:  0.9722222222222222

                   0         1
precision   0.969697  1.000000
recall      1.000000  0.750000
f1          0.984615  0.857143
support    32.000000  4.000000



,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,0,0
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


In [27]:
check_row(1500)

original msg:
The town of Les Cayes needs food items to assure. . ( MESSAGE INCOMPLETE )

tokenized msg:
['town', 'le', 'cay', 'need', 'food', 'item', 'assur', 'messag', 'incomplet']

Accuracy:  0.9444444444444444

                   0         1
precision   1.000000  0.333333
recall      0.942857  1.000000
f1          0.970588  0.500000
support    35.000000  1.000000



,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,0,1
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))],
 'verbose': True,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf

In [29]:
# parameters to tune
parameters = {
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators':[10, 50, 100, 150, 200, 500],
    'clf__estimator__learning_rate':[1.0,1.5,2.0]
}

# scorer using the average_f1
scorer = make_scorer(average_f1)

# Grid Search object
cv = GridSearchCV(pipeline, param_grid=parameters, scoring = scorer, verbose = 2, n_jobs = -1)

In [30]:
# fit the gridsearch object:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 130.7min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 182.2min finished


[Pipeline] .............. (step 1 of 3) Processing vect, total=  17.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 4.2min


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))],
                                verbose=True),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [1.0, 1.5, 2.0],
                         'clf__estimator__n_estimators': [10, 50, 100, 150, 200,
                                                          500],
                         'tfidf__use_idf': [True, False]},
             scoring=make_scorer(average_f1), verbose=2)

In [31]:
# see the best estimator
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=200,
                                                                    random_state=42)))],
         verbose=True)

In [32]:
print("Best score (Stratified):", cv.best_score_)
pd.concat([pd.DataFrame(cv.cv_results_["params"]),
           pd.DataFrame(cv.cv_results_["rank_test_score"],
                        columns=["Rank"]),
           pd.DataFrame(cv.cv_results_["mean_test_score"], 
                        columns=["f1-score"])],axis=1)

Best score (Stratified): 0.6920526449379663


,clf__estimator__learning_rate,clf__estimator__n_estimators,tfidf__use_idf,Rank,f1-score
0,1.0,10,True,22,0.671567
1,1.0,10,False,23,0.671118
2,1.0,50,True,6,0.688661
3,1.0,50,False,9,0.686005
4,1.0,100,True,3,0.690513
5,1.0,100,False,7,0.688122
6,1.0,150,True,2,0.691816
7,1.0,150,False,4,0.690070
8,1.0,200,True,1,0.692053
9,1.0,200,False,5,0.689572


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
Y_pred_cv = cv.predict(X_test) 

In [34]:
print_classification_report(Y_test, Y_pred_cv)

Category:  related
              precision    recall  f1-score   support

           0       0.65      0.41      0.50      1553
           1       0.83      0.93      0.88      5001

    accuracy                           0.81      6554
   macro avg       0.74      0.67      0.69      6554
weighted avg       0.79      0.81      0.79      6554

Category:  request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      5461
           1       0.74      0.56      0.64      1093

    accuracy                           0.89      6554
   macro avg       0.83      0.76      0.79      6554
weighted avg       0.89      0.89      0.89      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.12      0.03      0.05        32

    accuracy                           0.99      6554
   macro avg       0.56      0.52      0.52      6554
weighted avg       

Category:  other_weather
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6235
           1       0.42      0.17      0.24       319

    accuracy                           0.95      6554
   macro avg       0.69      0.58      0.61      6554
weighted avg       0.93      0.95      0.94      6554

Category:  direct_report
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      5289
           1       0.67      0.43      0.52      1265

    accuracy                           0.85      6554
   macro avg       0.77      0.69      0.72      6554
weighted avg       0.83      0.85      0.84      6554

-------------------------------------------------------
Mean accuracy score: 0.9471
Mean f1-score (macro): 0.6911


In [37]:
print("Naive accuracy: ", average_accuracy(Y_test, naive_pred))
print("Base model accuracy: ", average_accuracy(Y_test, Y_pred))
print("Optimized model accuracy: ", average_accuracy(Y_test, Y_pred_cv))

Naive accuracy:  0.9271691587834401
Base model accuracy:  0.947919845387041
Optimized model accuracy:  0.9470552334452242


In [64]:
print("Naive precision: ", average_precision(Y_test, naive_pred.to_numpy()))
print("Base model precision: ", average_precision(Y_test, Y_pred))
print("Optimized model precision: ", average_precision(Y_test, Y_pred_cv))

Naive precision:  0.47747346828060905
Base model precision:  0.7637077465683915
Optimized model precision:  0.7430995100960291


In [65]:
print("Naive recall: ", average_recall(Y_test, naive_pred.to_numpy()))
print("Base model recall: ", average_recall(Y_test, Y_pred))
print("Optimized model recall: ", average_recall(Y_test, Y_pred_cv))

Naive recall:  0.5138888888888888
Base model recall:  0.6550423293297105
Optimized model recall:  0.6659134870330656


In [38]:
print("Naive f1-score: ", average_f1(Y_test, naive_pred.to_numpy()))
print("Base model f1-score: ", average_f1(Y_test, Y_pred))
print("Optimized model f1-score: ", average_f1(Y_test, Y_pred_cv))

Naive f1-score:  0.49381425354511244
Base model f1-score:  0.6837673978011033
Optimized model f1-score:  0.6910561040580417


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [39]:
# check the shape of the CountVectorizer input:
vect_check = CountVectorizer(tokenizer=tokenize).fit_transform(X_test)

In [40]:
vect_check.shape

(6554, 12153)

The full test set has over 10K unique words. I'll apply the LSA (TrucatedSVD) to see if dimensionality reduction can improve fit

In [41]:
tfidf_check = TfidfTransformer().fit_transform(vect_check)

In [42]:
tfidf_check.shape

(6554, 12153)

In [44]:
svd_check = TruncatedSVD(n_components=100, random_state=42).fit_transform(tfidf_check)

In [45]:
svd_check.shape

(6554, 100)

In [46]:
pipeline_alt = Pipeline([
        ('vect' , CountVectorizer(tokenizer=tokenize)),
        ('tfidf' , TfidfTransformer()),
        ('svd',TruncatedSVD(n_components=100, random_state=42)),
        ('clf' , MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
], verbose = True)

In [47]:
pipeline_alt.fit(X_train, Y_train)

[Pipeline] .............. (step 1 of 4) Processing vect, total=  16.6s
[Pipeline] ............. (step 2 of 4) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 4) Processing svd, total=   1.9s
[Pipeline] ............... (step 4 of 4) Processing clf, total=10.0min


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023ED2FCE678>)),
                ('tfidf', TfidfTransformer()),
                ('svd', TruncatedSVD(n_components=100, random_state=42)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))],
         verbose=True)

In [48]:
Y_pred_alt = pipeline_alt.predict(X_test) 

In [49]:
print_classification_report(Y_test, Y_pred_alt)

Category:  related
              precision    recall  f1-score   support

           0       0.68      0.33      0.44      1553
           1       0.82      0.95      0.88      5001

    accuracy                           0.80      6554
   macro avg       0.75      0.64      0.66      6554
weighted avg       0.79      0.80      0.78      6554

Category:  request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5461
           1       0.80      0.50      0.62      1093

    accuracy                           0.90      6554
   macro avg       0.85      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       

C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Category:  refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6331
           1       0.00      0.00      0.00       223

    accuracy                           0.97      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.93      0.97      0.95      6554

Category:  death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6253
           1       0.84      0.16      0.27       301

    accuracy                           0.96      6554
   macro avg       0.90      0.58      0.63      6554
weighted avg       0.96      0.96    

In [50]:
print("Naive accuracy: ", average_accuracy(Y_test, naive_pred))
print("Base model accuracy: ", average_accuracy(Y_test, Y_pred))
print("Optimized model accuracy: ", average_accuracy(Y_test, Y_pred_cv))
print("Alternative base model accuracy: ", average_accuracy(Y_test, Y_pred_alt))

Naive accuracy:  0.9271691587834401
Base model accuracy:  0.947919845387041
Optimized model accuracy:  0.9470552334452242
Alternative base model accuracy:  0.9450886651069745


In [66]:
print("Naive precision: ", average_precision(Y_test, naive_pred.to_numpy()))
print("Base model precision: ", average_precision(Y_test, Y_pred))
print("Optimized model precision: ", average_precision(Y_test, Y_pred_cv))
print("Alternative base model precision: ", average_precision(Y_test, Y_pred_alt))

Naive precision:  0.47747346828060905
Base model precision:  0.7637077465683915
Optimized model precision:  0.7430995100960291
Alternative base model precision:  0.7290669056227514


In [67]:
print("Naive recall: ", average_recall(Y_test, naive_pred.to_numpy()))
print("Base model recall: ", average_recall(Y_test, Y_pred))
print("Optimized model recall: ", average_recall(Y_test, Y_pred_cv))
print("Alternative base model recall: ", average_recall(Y_test, Y_pred_alt))

Naive recall:  0.5138888888888888
Base model recall:  0.6550423293297105
Optimized model recall:  0.6659134870330656
Alternative base model recall:  0.5880465653061251


In [51]:
print("Naive f1-score: ", average_f1(Y_test, naive_pred.to_numpy()))
print("Base model f1-score: ", average_f1(Y_test, Y_pred))
print("Optimized model f1-score: ", average_f1(Y_test, Y_pred_cv))
print("Alternative base model f1-score: ", average_f1(Y_test, Y_pred_alt))

Naive f1-score:  0.49381425354511244
Base model f1-score:  0.6837673978011033
Optimized model f1-score:  0.6910561040580417
Alternative base model f1-score:  0.6041675471434715


### 9. Export your model as a pickle file

In [52]:
best_model = cv
pickle.dump(best_model, open('best_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.